In [30]:
GC.gc() # Clear up the memory when doing a "run all" in the notebook
using Plots
include("src/load_profiles.jl")
include("src/solver.jl");

In [31]:
filename = "profiles.csv"
YEARS = [2014, 2015, 2016, 2017, 2018]
if ! @isdefined(time_profiles) || Set(keys(time_profiles)) != Set(YEARS)
    time_profiles, wind_profiles, solar_profiles = load_profiles(filename, YEARS, true)
end;

print("Loaded profiles for years: ", keys(time_profiles), "\n")
print("Length of loaded profiles: ", length.(values(wind_profiles)), "\n")

Loaded profiles for years: Any[2018, 2014, 2017, 2016, 2015]
Length of loaded profiles: [8760, 8760, 8760, 8784, 8760]


In [32]:
# Default values
DEMAND = 1000. # Kg of H2
# Battery parameters
EBAT = 0.9 # per month discharge
FBAT = 100. # MW
COST_BAT = 250000 * 0.0002 # € / MWh
# Electrolyzer parameters
EELEC = 0.050 # MWh / Kg
COST_ELEC =  1200000 * 0.0004 # € / MW
CAPA_ELEC_UPPER = 1000 # MW
# Tank parameters
COST_TANK = 407. # € / Kg
# Grid parameters
PRICE_GRID = 1000. # € / MWh
PRICE_CURTAILING = 2000. # € / MWh;
PRICE_PENALITY = 0 # € / times changed
PRICE_PROD_CHANGE = 1 # € / kg of change in production level
# Renewable pricing, from https://atb.nrel.gov/electricity/2022/index
# For now : 20 year lifespan, no discount rate + O&M cost per year
COST_WIND = 1352 * 1000 / 20 + 43 * 1000 # € / MW
COST_SOLAR = 1233 * 1000 / 20 + 23 * 1000 # € / MW
# Upper bound on the battery capacity
CAPA_BAT_UPPER = 24 * DEMAND * EELEC # MW
#Year chosen for the simulation
YEAR = 2014;


In [33]:
prod_plot = plot(size=(1200, 500), legend=:topleft, xlabel="Time (h)", ylabel="Production (Kg)", title="Hydrogen, Demand : $DEMAND kg/h")
cons_plot = plot(size=(1200, 500), legend=:topleft, xlabel="Time (h)", ylabel="Consumption (MWh)", title="Electricity consumption")
level_bat = plot(size=(1200, 500), legend=:topleft, xlabel="Time (h)", ylabel="Battery level (MWh)", title="Battery level")
level_tank = plot(size=(1200, 500), legend=:topleft, xlabel="Time (h)", ylabel="Tank level (Kg)", title="Tank level");

In [34]:
wind_capacities = []
solar_capacities = []
battery_capacities = []
tank_capacities = []
electro_capacities = []

for year in keys(time_profiles)
    wind_profile = wind_profiles[year]
    solar_profile = solar_profiles[year]
    output = solve(
        wind_profile, solar_profile, # WIND_PROFILE, SOLAR_PROFILE
        DEMAND, # DEMAND
        missing, missing, # WIND_CAPA, SOLAR_CAPA, if set to missing, the solver will optimize the capacities
        missing, missing, missing, # BAT_SIZE, TANK_SIZE, ELEC_CAPA, if set to missing, the solver will optimize the capacities
        PRICE_GRID, PRICE_CURTAILING, # PRICE_GRID, PRICE_CURTAILING
        PRICE_PENALITY, PRICE_PROD_CHANGE, #PRICE_PENALITY, PRICE_PROD_CHANGE
        CAPA_BAT_UPPER, CAPA_ELEC_UPPER, # CAPA_BAT_UPPER, CAPA_ELEC_UPPER
        EBAT, FBAT, EELEC, # EBAT, FBAT, EELEC
        COST_ELEC, COST_BAT, COST_TANK, # COST_ELEC, COST_BAT, COST_TANK
        COST_WIND, COST_SOLAR, # COST_WIND, COST_SOLAR
        0., # No initial charge
        0., # No initial stock
        missing, # No final charge constraint
        missing, # No final stock constraint
    );
    # Extract the results
    battery_capa = trunc(Int64, output["battery_capa"])
    tank_capa = trunc(Int64, output["tank_capa"])
    electro_capa = trunc(Int64, output["electro_capa"])
    wind_capa = trunc(Int64, output["wind_capa"])
    solar_capa = trunc(Int64, output["solar_capa"])
    prod_out = output["prod"]
    charge_out = output["charge"]
    stock_out = output["stock"]
    elec_out = output["elecGrid"]
    curtailment_out = output["curtail"]
    consPPA_out = output["elecPPA"];
    # Plot the results
    plot!(prod_plot, prod_out, label="$year, Capa : $(electro_capa / EELEC) kg / h")
    plot!(cons_plot, elec_out, label="$year, Grid Import, Wind : $wind_capa MW")
    plot!(cons_plot, -curtailment_out, label="$year, Curtailment, Solar : $solar_capa MW")
    plot!(level_bat, charge_out, label="$year, Capa : $battery_capa MWh")
    plot!(level_tank, stock_out, label="$year, Capa : $tank_capa Kg")
    # Store the results
    push!(wind_capacities, wind_capa)
    push!(solar_capacities, solar_capa)
    push!(battery_capacities, battery_capa)
    push!(tank_capacities, tank_capa)
    push!(electro_capacities, electro_capa)
end;

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-25
Adding variables ... 
Adding constraints ... 


Optimizing ...


In [ ]:
prod_plot

In [ ]:
cons_plot

In [ ]:
level_bat

In [ ]:
level_tank

In [ ]:
# Print the capacities
println("Wind capacities : ", wind_capacities)
println("Average wind capacity : $(sum(wind_capacities) / length(wind_capacities)) MW")
println("Solar capacities : ", solar_capacities)
println("Average solar capacity : $(sum(solar_capacities) / length(solar_capacities)) MW")
println("Battery capacities : ", battery_capacities)
println("Average battery capacity : $(sum(battery_capacities) / length(battery_capacities)) MWh")
println("Tank capacities : ", tank_capacities)
println("Average tank capacity : $(sum(tank_capacities) / length(tank_capacities)) Kg")
println("Electrolyzer capacities : ", electro_capacities)
println("Average electrolyzer capacity : $(sum(electro_capacities) / length(electro_capacities)) Kg / h");